### Import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import xgboost
import warnings
warnings.filterwarnings("ignore")

### Load datasets

In [2]:
def clean_housing_price(df):
  # drop the extra columns and rows
  housing_price = df.drop(df.columns[range(62, len(df.columns))], axis=1)
  housing_price = housing_price.drop(list(range(385, 392)), axis=0)

  # build and insert month column and year column
  month = [date.split('-')[0] for date in housing_price['Mon-Yr']]
  year = [date.split('-')[1] for date in housing_price['Mon-Yr']]
  for idx, yr in enumerate(year):
    if yr[0] == '9':
      year[idx] = '19' + yr
    else:
      year[idx] = '20' + yr

  housing_price.insert(loc=1, value=month, column='Month')
  housing_price.insert(loc=2, value=year, column='Year')

  # format the price number notation
  for col in range(3, len(housing_price.columns)):
    for row in range(len(housing_price)):
      if not pd.isna(housing_price.iloc[row, col]):
        # remove the leading dollar sign and comma
        housing_price.iloc[row, col] = int(''.join(housing_price.iloc[row, col][1:].split(',')))
      else:
        # replace nan with None
        housing_price.iloc[row, col] = None

  return housing_price
  
housing_price_raw = pd.read_csv('data/housing_price.csv')
housing_price = clean_housing_price(housing_price_raw)
housing_price.head()

,Mon-Yr,Month,Year,CA,Alameda,Amador,Butte,Calaveras,Contra-Costa,Del Norte,...,Yuba,Unnamed: 53,Condo,LA Metro,Central Coast,Central Valley,Far North,Inland Empire,S.F. Bay Area,SoCal
0,Jan-90,Jan,1990,194952,226149,None,102143,None,None,None,...,None,NaN,141519,203390,None,None,None,None,227366,None
1,Feb-90,Feb,1990,196273,219306,None,83333,None,None,None,...,None,NaN,144965,211024,None,None,None,None,234739,None
2,Mar-90,Mar,1990,194856,225162,None,100000,None,None,None,...,None,NaN,141132,209286,None,None,None,None,235337,None
3,Apr-90,Apr,1990,196111,229333,None,108000,None,None,None,...,None,NaN,145707,210302,None,None,None,None,233178,None
4,May-90,May,1990,195281,232291,None,100000,None,None,None,...,None,NaN,146060,210148,None,None,None,None,235881,None


In [3]:
def clean_water_quality(df):
  # pick and rename only the related columns
  water_quality = df[['COUNTY_NAME', 'SAMPLE_DATE', 'PARAMETER', 'RESULT', 'REPORTING_LIMIT']]
  water_quality.columns = ['County', 'Date', 'Parameter', 'Result', 'Limit']

  # format the date into month and year
  month_dict = {
    '01': 'Jan',
    '02': 'Feb',
    '03': 'Mar',
    '04': 'Apr',
    '05': 'May',
    '06': 'Jun',
    '07': 'Jul',
    '08': 'Aug',
    '09': 'Sep',
    '10': 'Oct',
    '11': 'Nov',
    '12': 'Dec'
  }
  month = [month_dict[date.split('/')[0]] for date in water_quality['Date']]
  year = [date.split(' ')[0].split('/')[2] for date in water_quality['Date']]

  # add date and year column and drop the original date column
  water_quality.insert(loc=2, value=month, column='Month')
  water_quality.insert(loc=3, value=year, column='Year')
  water_quality.drop(columns='Date', axis=1, inplace=True)

  # drop the rows with NaN result or limit value
  # becasue we cannot tell if this a polluted water source
  water_quality.dropna(subset=['Result', 'Limit'], inplace=True)
  water_quality['Polluted'] = water_quality['Result'] > water_quality['Limit']

  # calculate the result to limit ratio and handle the division by zero case
  water_quality['Ratio'] = water_quality['Result'] / water_quality['Limit']
  water_quality['Ratio'].fillna(0, inplace=True)

  return water_quality

water_quality_raw = pd.read_csv('data/lab_results.csv')
water_quality = clean_water_quality(water_quality_raw)
water_quality.head()

,County,Month,Year,Parameter,Result,Limit,Polluted,Ratio
0,Alameda,May,1967,Conductance,3480.0,1.0,True,3480.0
1,Alameda,May,1967,Dissolved Boron,7.7,0.1,True,77.0
2,Alameda,May,1967,Dissolved Calcium,68.0,1.0,True,68.0
3,Alameda,May,1967,Dissolved Chloride,758.0,0.1,True,7580.0
4,Alameda,May,1967,Dissolved Magnesium,59.0,0.1,True,590.0


In [6]:
air_quality_file_prefix = 'data/annual_aqi_by_county_'
air_quality_dataset_list = []
for year in range(2000, 2022):
  air_quality_file_path = air_quality_file_prefix + str(year) + '.csv'
  curr_dataset = pd.read_csv(air_quality_file_path)
  curr_dataset = curr_dataset[curr_dataset['State'] == 'California']
  air_quality_dataset_list.append(curr_dataset)
air_quality = pd.concat(air_quality_dataset_list)
air_quality.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
63,California,Alameda,2000,366,293,59,11,2,1,0,209,70,39,2,129,182,0,53,0
64,California,Amador,2000,366,238,83,34,11,0,0,192,112,44,15,0,351,0,0,0
65,California,Butte,2000,361,226,88,40,7,0,0,174,108,44,16,41,260,0,25,19
66,California,Calaveras,2000,364,245,71,34,14,0,0,200,115,38,29,0,311,0,23,1
67,California,Colusa,2000,365,306,54,5,0,0,0,105,61,36,0,0,280,0,30,55


In [15]:
wells_data = pd.read_csv('data/wells_data.csv')
wells_data = wells_data.drop(['OID', 'API', 'LeaseName', 'WellNumber', 'WellTypeLa',
                              'OperatorCo', 'OperatorNa', 'FieldName',
                              'AreaName', 'District', 'Section',
                              'Township', 'Range', 'BaseMeridi',
                              'GISSource', 'WellSymbol'], axis = 1)
wells_data.head()

,WellStatus,WellType,CountyName,Latitude,Longitude,isConfiden,isDirectio,SpudDate
0,Idle,DG,Glenn,39.449158,-121.946289,N,N,02/17/1988
1,Idle,DG,Glenn,39.440640,-121.951889,N,N,12/07/1999
2,Idle,DG,Glenn,39.447174,-121.961273,N,N,05/13/2000
3,Idle,DG,Glenn,39.443230,-121.951042,N,N,11/06/2002
4,Plugged,DH,Glenn,39.438267,-121.947418,N,N,06/11/1959


### Preprocess datasets

In [4]:
# unpivot housing price dataset
# melt from horizontal to vertical format
housing_price['Mon-Yr'] = housing_price['Month'] + '-' + housing_price['Year']
hp = housing_price.drop(['Month', 'Year', 'CA'], axis=1)
hp = pd.melt(hp, id_vars=['Mon-Yr'])
hp.columns = ['mon-yr', 'county', 'price']
hp = hp.replace(r'^\s*$', np.nan, regex=True)
hp.dropna(inplace=True)
hp.drop_duplicates(inplace=True)
hp.head()

,mon-yr,county,price
0,Jan-1990,Alameda,226149.0
1,Feb-1990,Alameda,219306.0
2,Mar-1990,Alameda,225162.0
3,Apr-1990,Alameda,229333.0
4,May-1990,Alameda,232291.0


In [5]:
water_quality['mon-yr'] = water_quality['Month'] + '-' + water_quality['Year']
wq = water_quality.drop(['Month', 'Year', 'Limit'], axis=1)
# wq = wq[wq['Parameter'].isin({'Total Copper', 'Total Chromium', 'Total Cadmium', 'Total Lead'})]
wq = wq.replace(r'^\s*$', np.nan, regex=True)
wq.dropna(inplace=True)
wq.drop_duplicates(inplace=True)
wq.head(3)

,County,Parameter,Result,Polluted,Ratio,mon-yr
0,Alameda,Conductance,3480.0,True,3480.0,May-1967
1,Alameda,Dissolved Boron,7.7,True,77.0,May-1967
2,Alameda,Dissolved Calcium,68.0,True,68.0,May-1967


In [147]:
wq.shape

(3133296, 6)

In [13]:
aq = air_quality.drop(['State'], axis=1)
aq = aq.replace(r'^\s*$', np.nan, regex=True)
aq.dropna(inplace=True)
aq.drop_duplicates(inplace=True)
aq.head(3)

,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
63,Alameda,2000,366,293,59,11,2,1,0,209,70,39,2,129,182,0,53,0
64,Amador,2000,366,238,83,34,11,0,0,192,112,44,15,0,351,0,0,0
65,Butte,2000,361,226,88,40,7,0,0,174,108,44,16,41,260,0,25,19


In [95]:
ow = wells_data[['CountyName', 'SpudDate', 'WellStatus']]
ow = ow.replace(r'^\s*$', np.nan, regex=True)
ow.fillna('2022', inplace=True)
ow['SpudAge'] = ow['SpudDate'].apply(lambda x: max(0, 2022 - int(x[-4:])))
avg_spud_age = int(ow[ow['SpudAge'] > 0].mean())
ow['SpudAge'] = ow['SpudAge'].apply(lambda x: avg_spud_age if x == 0 else x)
ow['WellStatus'] = ow['WellStatus'].apply(lambda x: 1 if x == 'Active' else 0)
ow = ow[['CountyName', 'SpudAge', 'WellStatus']].groupby('CountyName').mean().reset_index()
ow.columns = ['CountyName', 'SpudAge', 'ActiveWellRatio']
ow.head(3)

,CountyName,SpudAge,ActiveWellRatio
0,Alameda,60.547368,0.063158
1,Amador,62.000000,0.000000
2,Butte,53.299213,0.086614


In [6]:
# join four tables together
hp_wq = hp.merge(wq, left_on=['county', 'mon-yr'], right_on=['County', 'mon-yr'])
# hp_wq['house_price_age'] = hp_wq['mon-yr'].apply(lambda x: max(0, 2022 - int(x[-4:])))
hp_wq.drop(['County'], axis=1, inplace=True)                                   

In [7]:
hp_wq.head()

,mon-yr,county,price,Parameter,Result,Polluted,Ratio
0,Jan-1990,Alameda,226149.0,Bromodichloromethane,120.0,True,12.0
1,Jan-1990,Alameda,226149.0,Bromoform,0.0,False,0.0
2,Jan-1990,Alameda,226149.0,Chloroform,360.0,True,36.0
3,Jan-1990,Alameda,226149.0,Color,25.0,True,5.0
4,Jan-1990,Alameda,226149.0,Conductance,849.0,True,849.0


In [187]:
categorical_features = ['county', 'Parameter']
ohe = OneHotEncoder()
transformed_hp_wq = ohe.fit_transform(hp_wq[categorical_features])

In [188]:
transformed_hp_wq = pd.DataFrame(transformed_hp_wq.toarray(), columns = np.concatenate((ohe.categories_[0], ohe.categories_[1])))

In [191]:
hp_wq = pd.concat([hp_wq, transformed_hp_wq], axis=1)
hp_wq.drop(['county', 'Parameter'], axis=1, inplace = True)

In [194]:
hp_wq.to_csv('/Users/ryanyang/Desktop/USC/DSCI_560/project/data/house_price_water_quality.csv', index=False)

In [200]:
hp_aq = hp.merge(aq, left_on='county', right_on='County')
hp_aq['house_price_age'] = hp_aq['mon-yr'].apply(lambda x: max(0, 2022 - int(x[-4:])))

In [206]:
ohe = OneHotEncoder()
transformed_hp_aq = ohe.fit_transform(hp_aq[['county']])
hp_aq[ohe.categories_[0]] = transformed_hp_aq.toarray()

In [208]:
hp_aq.drop(['mon-yr', 'County', 'county'], axis=1, inplace=True)

In [210]:
hp_aq.shape

(331998, 68)

In [211]:
hp_aq.to_csv('/Users/ryanyang/Desktop/USC/DSCI_560/project/data/house_price_air_quality.csv', index=False)

In [217]:
hp_ow = hp.merge(ow, left_on='county', right_on='CountyName')
hp_ow['house_price_age'] = hp_ow['mon-yr'].apply(lambda x: max(0, 2022 - int(x[-4:])))

In [218]:
ohe = OneHotEncoder()
transformed_hp_ow = ohe.fit_transform(hp_ow[['county']])
hp_ow[ohe.categories_[0]] = transformed_hp_ow.toarray()

In [221]:
hp_ow.drop(['mon-yr', 'county', 'CountyName'], axis=1, inplace=True)

In [223]:
hp_ow.to_csv('/Users/ryanyang/Desktop/USC/DSCI_560/project/data/house_price_oil_wells.csv', index=False)

### Train / Test split

In [ ]:
y = final_df['price']
X = final_df.drop(['price'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

### Baseline Model training

In [ ]:
xgb = xgboost.XGBClassifier().fit(X_train, y_train)
xgb_predict = xgb.predict(X_test)
accuracy_score(y_test, xgb_predict)

ValueError: ignored